In [1]:
### load package
import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np
import gurobipy as gp
import time
model_sanitized_path = 'C:/Users/52427/Documents/Cases/Case11_Baixiang/Baixiang_supply_chain_optimization/model_sanitized/'
from datetime import date

### Load input tables

In [2]:
file_name = model_sanitized_path + 'model_input_20220615_EN.xlsx'
xls = pd.ExcelFile(file_name)

customers_df = pd.read_excel(xls, 'Input_CustomerList')

factories_df = pd.read_excel(xls, 'Input_FactoryList')

products_df = pd.read_excel(xls, 'Input_ProductList')

demand_df = pd.read_excel(xls, 'Input_CustomerDemand')

trans_policies_df = pd.read_excel(xls, 'Input_TransportationPolicy')

prod_policies_df = pd.read_excel(xls, 'Input_CapacityByLineByProduct')

prod_line_policies_df = pd.read_excel(xls, 'Input_CapacityByLine')

factory_var_cost_df = pd.read_excel(xls, 'Input_FactoryVariableCost')

factory_fixed_cost_df = pd.read_excel(xls, 'Input_FactoryFixedCost')

inv_df = pd.read_excel(xls, 'Input_InventoryPolicy')

### load baseline data tables
hist_outbound_trans_df = pd.read_excel(xls, sheet_name='Input_HistShipmentFactoryToCust')

hist_inbound_trans_df = pd.read_excel(xls, sheet_name='Input_HistShipmentAmongFactory')

hist_prod_df = pd.read_excel(xls, sheet_name='Input_HistProduction')

hist_inv_df = pd.read_excel(xls, sheet_name='Input_HistInventoryLevel')

xls.close()

In [3]:
### print data info
print('# customers:', customers_df.shape[0])
print('# factories:', factories_df.shape[0])
print('# products:', products_df.shape[0])
print('# customer-product combinations in demand table:', demand_df.shape[0])
print('# customers in demand table:', len(demand_df['CustomerName'].unique()))
print('# products in demand table:', len(demand_df['ProductName'].unique()))
print('# all lanes:', trans_policies_df.shape[0])
print('# factories in transportation table:', trans_policies_df[['Origin']].drop_duplicates().shape[0])
print('# destinations in transportation table:', trans_policies_df[['Destination']].drop_duplicates().shape[0])
print('# line-product combinations:',prod_policies_df.shape[0])
print('# lines:',prod_line_policies_df.shape[0])
print('# factories in factory fixed cost table:', factory_fixed_cost_df.shape[0])
print('# products in factory variable cost table:', len(factory_var_cost_df[['ProductName']].drop_duplicates()))

### print baseline data info
print('# sites to customers:', hist_outbound_trans_df.shape[0])
print('# sites to sites:', hist_inbound_trans_df.shape[0])
print('# production combs:', hist_prod_df.shape[0])

# customers: 293
# factories: 9
# products: 61
# customer-product combinations in demand table: 34115
# customers in demand table: 293
# products in demand table: 61
# all lanes: 2754
# factories in transportation table: 9
# destinations in transportation table: 307
# line-product combinations: 187
# lines: 59
# factories in factory fixed cost table: 9
# products in factory variable cost table: 61
# sites to customers: 34932
# sites to sites: 506
# production combs: 1642


### Preprocess

In [4]:
### create lists of basic model elements
customers = [x for x in customers_df['CustomerName']]

factories = [x for x in factories_df['FactoryName']]

products = [x for x in products_df['ProductName']]

months = range(1, 13)

In [5]:
### customer demand
cust_demand = demand_df.groupby(['CustomerName', 'ProductName', 'Period'])['CustomerDemand'].sum().to_dict()

### production capacity
line_product_cap = prod_policies_df.groupby(['FactoryName', 'ProductLine', 'ProductName'])['Capacity'].sum().to_dict()

line_overall_cap = prod_line_policies_df.groupby(['FactoryName', 'ProductLine'])['Capacity'].sum().to_dict()

### factory-attached warehouse storage capacity
factory_storage_cap = factories_df.groupby(['FactoryName'])['StorageCapacity'].max().to_dict()

### factory-attached warehouse handling capacity
factory_handling_cap = factories_df.groupby(['FactoryName'])['HandlingCapacity'].max().to_dict()

### factory variable cost (including conversion and handling)
factory_var_cost = factory_var_cost_df.groupby(['FactoryName', 'ProductName'])['VariableCost'].sum().to_dict()

### factory fixed cost
factory_fixed_cost = factory_fixed_cost_df.groupby(['FactoryName'])['FixedOperatingCost'].sum().to_dict()

### transportation cost
transp_cost = trans_policies_df.groupby(['Origin', 'Destination'])['Price'].max().to_dict()

### transportation distance
transp_dist = trans_policies_df.groupby(['Origin', 'Destination'])['Distance'].max().to_dict()

### unit product cubic meters
product_cubic = products_df.groupby(['ProductName'])['Volume'].max().to_dict()

### inventory turns
inv_turns = inv_df.groupby(['FactoryName'])['InventoryTurns'].max().to_dict()

### initial inventory levels
min_initial_inv = inv_df.groupby(['FactoryName'])['MinHistoricalInventory'].max().to_dict()
max_initial_inv = inv_df.groupby(['FactoryName'])['MaxHistoricalInventory'].max().to_dict()

### inventory levels
min_inv = hist_inv_df.groupby(['FactoryName','Period'])['MinHistoricalInventory'].max().to_dict()
max_inv = hist_inv_df.groupby(['FactoryName','Period'])['MaxHistoricalInventory'].max().to_dict()

In [6]:
### historical factory-to-customer flows
hist_fc_flows = hist_outbound_trans_df.groupby(
    ['Origin', 'Destination', 'ProductName', 'Period'])['HistoricalShipment'].sum().to_dict()

### historical inter-factory flows
hist_ff_flows = hist_inbound_trans_df.groupby(
    ['Origin', 'Destination', 'ProductName', 'Period'])['HistoricalShipment'].sum().to_dict()

### historical production
hist_prod_flows = hist_prod_df.groupby(
    ['FactoryName', 'ProductLine', 'ProductName', 'Period'])['HistoricalProduction'].sum().to_dict()

In [7]:
### generate index sets
# ff_lanes = gp.tuplelist([(o, d, p, m) for o in factories for d in factories if d!=o for p in products for m in months]) 
ff_lanes = gp.tuplelist(hist_ff_flows.keys())

fc_lanes = gp.tuplelist([(o, d, p, m) for o in factories for (d, p, m) in cust_demand]) 

prod_policies_month_df = pd.merge(prod_policies_df[['FactoryName', 'ProductLine', 'ProductName']].assign(temp=1), 
                                  pd.DataFrame(months, columns =['Period']).assign(temp=1), on='temp').drop('temp', axis=1)
prod_policies = gp.tuplelist(zip(prod_policies_month_df['FactoryName'], prod_policies_month_df['ProductLine'], 
                                 prod_policies_month_df['ProductName'], prod_policies_month_df['Period']))
factory_lines = list(zip(prod_policies_month_df['FactoryName'], prod_policies_month_df['ProductLine']))

inv_basis = gp.tuplelist([(f, p, m) for f in factories for p in products for m in list(months) + [max(months)+1]])

In [8]:
### dummy source to supply quantity deficit
dsrc_lanes = gp.tuplelist([('dsrc', d, p, m) for d in factories for p in products for m in months])

### dummy sink to absorb quantity excess
dsnk_lanes = gp.tuplelist([(o, 'dsnk', p, m) for o in factories for p in products for m in months])

### per unit penalty cost of using dummy lanes
dsrc_pen = np.ceil(max(factory_var_cost.values())*10 + max(transp_cost.values())*10)
dsnk_pen = 1

### Scenario setting

In [9]:
# scenarios matrix
scenario_list = ['Baseline2021','Unconstrainted','CloseOneFactory','Unconstrainted+RemoveStorageCons','Unconstrainted+RemoveHandlingCons']
constr_setting = {'factory_handling_cap':[False, True, True, True, False], 
                  'factory_storage_cap':[False, True, True, False, True],
                  'hist_fc':[True, False, False, False, False], 
                  'hist_ff':[True, False, False, False, False], 
                  'hist_prod':[True, False, False, False, False], 
                  'hist_no_prod':[True, False, False, False, False], 
                  'fix_factories_open':[True, True, True, True, True],
                  'num_factories_open':[9, 9, 8, 9, 9], 
                  'deact_dummy':[False, True, True, True, True],
                  'initial_inv': [True, True, True, True, True]}
scenario_matrix = pd.DataFrame(constr_setting, index=scenario_list)
scenario_matrix

,factory_handling_cap,factory_storage_cap,hist_fc,hist_ff,hist_prod,hist_no_prod,fix_factories_open,num_factories_open,deact_dummy,initial_inv
Baseline2021,False,False,True,True,True,True,True,9,False,True
Unconstrainted,True,True,False,False,False,False,True,9,True,True
CloseOneFactory,True,True,False,False,False,False,True,8,True,True
Unconstrainted+RemoveStorageCons,True,False,False,False,False,False,True,9,True,True
Unconstrainted+RemoveHandlingCons,False,True,False,False,False,False,True,9,True,True


### Network Optimization Model

In [10]:
today = date.today()
today = str(today).replace('-','')

#scenario = 'Baseline2021'
#scenario = 'Unconstrainted'
#scenario = 'CloseOneFactory'
#scenario = 'Unconstrainted+RemoveStorageCons'
scenario = 'Unconstrainted+RemoveHandlingCons'

In [11]:
time_start = time.time()

### create model
m = gp.Model('BXNetworkOpt')

### decision variables
ff_flow = m.addVars(ff_lanes, vtype=gp.GRB.CONTINUOUS, name='ff_flow') # inter-factory flows
fc_flow = m.addVars(fc_lanes, vtype=gp.GRB.CONTINUOUS, name='fc_flow') # factory-to-customer flows
prod_flow = m.addVars(prod_policies, vtype=gp.GRB.CONTINUOUS, name='prod_flow') # production volume
inv_level = m.addVars(inv_basis, vtype=gp.GRB.CONTINUOUS, name='inv_level') # beginning inventory level of each month
f_open = m.addVars(factories, vtype=gp.GRB.BINARY, name='f_open') # factory open or not

dsrc_flow = m.addVars(dsrc_lanes, vtype=gp.GRB.CONTINUOUS, name='dsrc_flow') # flow out of dummy source
dsnk_flow = m.addVars(dsnk_lanes, vtype=gp.GRB.CONTINUOUS, name='dsnk_flow') # flow into dummy sink

m.update()

### objective function
tot_factory_var_cost = gp.quicksum(prod_flow[(f, l, p, m)]*factory_var_cost[(f, p)] for (f, l, p, m) in prod_policies) 
tot_factory_fixed_cost = gp.quicksum(f_open[f]*factory_fixed_cost[f] for f in factories)
tot_ff_transp_cost = gp.quicksum(ff_flow[(o, d, p, m)]*product_cubic[p]*transp_cost[(o, d)] for (o, d, p, m) in ff_lanes)
tot_fc_transp_cost = gp.quicksum(fc_flow[(o, d, p, m)]*product_cubic[p]*transp_cost[(o, d)] for (o, d, p, m) in fc_lanes)
tot_dlanes_pen_cost = dsrc_flow.sum('*', '*', '*', '*')*dsrc_pen + dsnk_flow.sum('*', '*', '*', '*')*dsnk_pen
tot_cost = tot_factory_var_cost + tot_factory_fixed_cost + tot_ff_transp_cost + tot_fc_transp_cost + tot_dlanes_pen_cost

m.setObjective(tot_cost, gp.GRB.MINIMIZE)

### demand satisfaction
m.addConstrs(
    (fc_flow.sum('*', c, p, m) == cust_demand[(c, p, m)] for (c, p, m) in cust_demand.keys()), 'customer_demand'
)

### flow balance
m.addConstrs(
    (inv_level[(f, p, m)] + ff_flow.sum('*', f, p, m) + prod_flow.sum(f, '*', p, m) + dsrc_flow[('dsrc', f, p, m)] == 
     ff_flow.sum(f, '*', p, m) + fc_flow.sum(f, '*', p, m) + inv_level[(f, p, m+1)] + dsnk_flow[(f, 'dsnk', p, m)]
     for f in factories for p in products for m in months), 'flow_balance'
)

### production capacity by line and product
m.addConstrs(
    (prod_flow[(f, l, p, m)] <= line_product_cap[(f, l, p)]*f_open[f] for (f, l, p, m) in prod_policies), 'line_product_cap'
)

### production capacity by line overall
m.addConstrs(
    (prod_flow.sum(f, l, '*', m) <= line_overall_cap[(f, l)] for f, l in factory_lines for m in months), 'line_overall_cap'
)

### factory-attached warehouse handling capacity
if scenario_matrix.loc[scenario, 'factory_handling_cap']:
    m.addConstrs(
        (ff_flow.sum(f, '*', '*', m) + fc_flow.sum(f, '*', '*', m) 
         <= factory_handling_cap[f]*f_open[f] for f in factories for m in months), 'factory_handling_cap'
    )

### factory-attached warehouse storage capacity
if scenario_matrix.loc[scenario, 'factory_storage_cap']:
    m.addConstrs(
        (inv_level.sum(f, '*', m) 
         <= factory_storage_cap[f]*f_open[f] for f in factories for m in months), 'factory_storage_cap'
    )

### historical factory-to-customer flows
if scenario_matrix.loc[scenario, 'hist_fc']:
    m.addConstrs(
        (fc_flow[(o, d, p, m)] == hist_fc_flows[(o, d, p, m)] for (o, d, p, m) in hist_fc_flows.keys()), 'hist_fc'
    )

### historical inter-factory flows
if scenario_matrix.loc[scenario, 'hist_ff']:
    m.addConstrs(
        (ff_flow[(o, d, p, m)] == hist_ff_flows[(o, d, p, m)] for (o, d, p, m) in hist_ff_flows.keys()), 'hist_ff'
    )

### historical production
if scenario_matrix.loc[scenario, 'hist_prod']:
    m.addConstrs(
        (prod_flow[(f, l, p, m)] == hist_prod_flows[(f, l, p, m)] for (f, l, p, m) in hist_prod_flows.keys()), 'hist_prod'
    )

### historical no production
if scenario_matrix.loc[scenario, 'hist_no_prod']:
    m.addConstrs(
        (prod_flow[(f, l, p, m)] == 0 for (f, l, p, m) in [x for x in prod_policies if x not in hist_prod_flows.keys()]), 
        'hist_no_prod'
    )

### number of factories open
if scenario_matrix.loc[scenario, 'fix_factories_open']:
    m.addConstr(
        (f_open.sum('*') == scenario_matrix.loc[scenario, 'num_factories_open']), 'num_factories_open'
    )

### activate / deactivate dummy source and sink
if scenario_matrix.loc[scenario, 'deact_dummy']:
    m.addConstrs((dsrc_flow[o, d, p, m] == 0 for (o, d, p, m) in dsrc_lanes), 'deactivate_source')
    m.addConstrs((dsnk_flow[o, d, p, m] == 0 for (o, d, p, m) in dsnk_lanes), 'deactivate_sink')


### zero inital inventory
if scenario_matrix.loc[scenario, 'initial_inv']:
    m.addConstrs(
        (inv_level[f, p, 1] == 0 for f in factories for p in products), 'intial_inv'
    )


time_build_end = time.time()

### solve the model
m.optimize()
time_solve_end = time.time()

print()
print('optimal' if m.status==gp.GRB.OPTIMAL else 'infeasible')
print('model build time:', round(time_build_end - time_start), 's')
print('model solve time:', round(time_solve_end - time_build_end), 's')
tot_oprt_cost = tot_factory_var_cost.getValue() + tot_factory_fixed_cost.getValue() \
                + tot_fc_transp_cost.getValue() + tot_ff_transp_cost.getValue()
print('total operating cost:', tot_oprt_cost)

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "473023-DevTestPoolforBaixiangNetOpt"
Waiting for cloud server to start (pool 473023-DevTestPoolforBaixiangNetOpt)...
Starting...
Starting...
Starting...
Compute Server job ID: a205c87b-69a4-415b-9141-348e5dc722b2
Capacity available on '473023-DevTestPoolforBaixiangNetOpt' cloud pool - connecting...
Established HTTPS encrypted connection
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Gurobi Compute Server Worker version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 83709 rows, 330107 columns and 773284 nonzeros
Model fingerprint: 0x7b1acd6f
Variable types: 330098 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+05]
  Objective range  [1e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 9e+05]
Presolve removed 54593 rows and 151523 columns
Presol

### Optimization Results

In [12]:
fc_flow_res = []

for i in fc_lanes:
    if fc_flow[i].x > 0:
        var_output = {
            'Origin': i[0],
            'Destination': i[1],
            'ProductName': i[2],
            'Period': i[3],
            'Quantity': fc_flow[i].x
        }
        fc_flow_res.append(var_output)

fc_flow_res_df = pd.DataFrame.from_records(fc_flow_res)

len(fc_flow_res_df) == len(hist_outbound_trans_df)

False

In [13]:
ff_flow_res = []

for i in ff_lanes:
    if ff_flow[i].x > 0:
        var_output = {
            'Origin': i[0],
            'Destination': i[1],
            'ProductName': i[2],
            'Period': i[3],
            'Quantity': ff_flow[i].x
        }
        ff_flow_res.append(var_output)

ff_flow_res_df = pd.DataFrame.from_records(ff_flow_res)

len(ff_flow_res_df) == len(hist_inbound_trans_df)

False

In [14]:
prod_flow_res = []

for i in prod_policies:
    if prod_flow[i].x > 0:
        var_output = {
            'FactoryName': i[0],
            'ProductLine': i[1],
            'ProductName': i[2],
            'Period': i[3],
            'Quantity': prod_flow[i].x
        }
        prod_flow_res.append(var_output)
        
prod_flow_res_df = pd.DataFrame.from_records(prod_flow_res)

len(prod_flow_res_df) == len(hist_prod_df)

False

In [15]:
inv_level_res = []

for i in inv_basis:
    if inv_level[i].x > 0:
        var_output = {
            'FactoryName': i[0],
            'ProductName': i[1],
            'Period': i[2],
            'Quantity': inv_level[i].x
        }
        inv_level_res.append(var_output)
        
inv_level_res_df = pd.DataFrame.from_records(inv_level_res)

inv_level_res_df.head()

,FactoryName,ProductName,Period,Quantity
0,SITE_1,PROD_9,3,6564.0
1,SITE_1,PROD_9,5,52.0
2,SITE_1,PROD_11,3,21617.0
3,SITE_1,PROD_11,4,19107.0
4,SITE_1,PROD_11,5,8312.0


In [16]:
dummy_flow_res = [] 

for i in dsrc_lanes:
    if dsrc_flow[i].x > 0:
        var_output = {
            'Category': 'dummy source',
            'FactoryName': i[1],
            'ProductName': i[2],
            'Period': i[3],
            'Quantity': dsrc_flow[i].x
        }
        dummy_flow_res.append(var_output)
        
for i in dsnk_lanes:
    if dsnk_flow[i].x > 0:
        var_output = {
            'Category': 'dummy sink',
            'FactoryName': i[0],
            'ProductName': i[2],
            'Period': i[3],
            'Quantity': dsnk_flow[i].x
        }
        dummy_flow_res.append(var_output) 
        
dummy_flow_res_df = pd.DataFrame.from_records(dummy_flow_res)

if len(dummy_flow_res_df)>1:
    # dummy sink flows could be due to e-commerce which is in scope for production data but out of scope for distribution data
    # dummy source flows need to be cleared though; could be due to product name misalignment?
    # dummy_flow_res_df[(dummy_flow_res_df['Category']=='dummy source')&(dummy_flow_res_df['Quantity']>=1)].sort_values('Volume', ascending=False).head()
    print(dummy_flow_res_df.groupby(['Category'])[['Quantity']].sum())
else:
    print('no dummy flows')

no dummy flows


In [17]:
f_open_res = []

for i in factories:
    var_output = {
        'FactoryName': i,
        'FactoryStatus': f_open[i].x
    }
    f_open_res.append(var_output)
        
f_open_res_df = pd.DataFrame.from_records(f_open_res)

f_open_res_df

,FactoryName,FactoryStatus
0,SITE_1,1.0
1,SITE_2,1.0
2,SITE_3,1.0
3,SITE_4,1.0
4,SITE_5,1.0
5,SITE_6,1.0
6,SITE_7,1.0
7,SITE_8,1.0
8,SITE_9,1.0


### Prepare output tables

In [18]:
### factory to customer transportation cost
fc_trans_cost_sum_df = fc_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination'])\
    .merge(products_df, on = ['ProductName'])\
        .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['Price']})\
            .groupby(['Origin','ProductName','Period'])\
                .agg({'TransportationCost':'sum','Quantity':'sum'}).reset_index()\
                    .rename({'Origin':'FactoryName', 'TransportationCost':'TransportationCostFactoryToCustomer', 'Quantity':'QuantityFactoryToCustomer'},axis=1)

### factory to factory transportation cost
if ff_flow_res_df.shape[0] != 0:
    ff_trans_cost_sum_df = ff_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination'])\
        .merge(products_df, on = ['ProductName'])\
            .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['Price']})\
                 .groupby(['Destination','ProductName','Period'])\
                    .agg({'TransportationCost':'sum','Quantity':'sum'}).reset_index()\
                        .rename({'Destination':'FactoryName', 'TransportationCost':'TransportationCostAmongFactoryInflow', 'Quantity':'QuantityAmongFactoryInflow'},axis=1)
else:
    ff_trans_cost_sum_df = pd.DataFrame(columns=['FactoryName','Period','TransportationCostAmongFactoryInflow','QuantityAmongFactoryInflow'])

### factory variable cost
factory_var_cost_sum_df = prod_flow_res_df.merge(factory_var_cost_df, on = ['FactoryName','ProductName'])\
    .assign(**{"FactoryVariableCost": lambda x: x['Quantity']*x['VariableCost']})\
            .groupby(['FactoryName','ProductName','Period'])\
                .agg({'FactoryVariableCost':'sum','Quantity':'sum'}).reset_index()\
                    .rename({'Quantity':'ProductionQuantity'},axis=1)

### factory fixed cost
factory_fixed_cost_sum_df = pd.merge(f_open_res_df, factory_fixed_cost_df, on = 'FactoryName').assign(**{'FactoryFixedOperatingCost': lambda x: x['FactoryStatus']*x['FixedOperatingCost']})

### factory level cost summary
factory_cost_sum_df = fc_trans_cost_sum_df.groupby(['FactoryName'])[['TransportationCostFactoryToCustomer', 'QuantityFactoryToCustomer']].sum().reset_index()\
    .merge(ff_trans_cost_sum_df.groupby(['FactoryName'])[['TransportationCostAmongFactoryInflow','QuantityAmongFactoryInflow']].sum().reset_index(), on = ['FactoryName'], how = 'left').fillna(0)\
        .merge(ff_flow_res_df.groupby(['Origin']).agg({'Quantity':'sum'}).reset_index().rename({'Origin':'FactoryName', 'Quantity':'QuantityAmongFactoryOutflow'},axis=1), on = ['FactoryName'], how = 'left').fillna(0)\
            .merge(factory_var_cost_sum_df.groupby(['FactoryName'])[['FactoryVariableCost','ProductionQuantity']].sum().reset_index(), on = ['FactoryName'])\
                .merge(factory_fixed_cost_sum_df[['FactoryName','FactoryFixedOperatingCost']], on = ['FactoryName'])\
                    .merge(factories_df[['FactoryName','StorageCapacity','HandlingCapacity']].assign(HandlingCapacity = lambda x: x['HandlingCapacity']*12).rename({'HandlingCapacity':'AnnualHandlingCapacity'},axis=1), on = ['FactoryName'])\
                        .merge(inv_df[['FactoryName','InventoryTurns']].rename({'InventoryTurns':'AnnualInventoryTurns'},axis=1), on = ['FactoryName'])\
                            .merge(prod_line_policies_df.groupby(['FactoryName']).agg(Capacity = ('Capacity', lambda x: x.sum() * 12)).reset_index().rename({'Capacity':'AnnualProductionCapacity'},axis=1), on = ['FactoryName'])\
                                .assign(**{'HandlingCapacityUtilization': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])/x['AnnualHandlingCapacity']})\
                                    .assign(**{'StorageCapacityUtilization': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])/(x['AnnualInventoryTurns']*x['StorageCapacity'])})\
                                        .assign(**{'ProductionCapacityUtilization': lambda x: x['ProductionQuantity']/x['AnnualProductionCapacity']})\
                                            .assign(**{'TransportationQuantityOutflow': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])})\
                                                .assign(**{'TransportationCost': lambda x: (x['TransportationCostFactoryToCustomer']+x['TransportationCostAmongFactoryInflow'])})

### factory + month level cost summary
factory_cost_monthly_sum_df = fc_trans_cost_sum_df.groupby(['FactoryName','Period'])[['TransportationCostFactoryToCustomer', 'QuantityFactoryToCustomer']].sum().reset_index()\
    .merge(ff_trans_cost_sum_df.groupby(['FactoryName','Period'])[['TransportationCostAmongFactoryInflow','QuantityAmongFactoryInflow']].sum().reset_index(), on = ['FactoryName','Period'], how = 'outer')\
        .merge(ff_flow_res_df.groupby(['Origin','Period']).agg({'Quantity':'sum'}).reset_index().rename({'Origin':'FactoryName', 'Quantity':'QuantityAmongFactoryOutflow'},axis=1), on = ['FactoryName','Period'], how = 'left').fillna(0)\
            .merge(factory_var_cost_sum_df.groupby(['FactoryName','Period'])[['FactoryVariableCost','ProductionQuantity']].sum().reset_index(), on = ['FactoryName','Period'], how = 'outer')\
                .merge(factory_fixed_cost_sum_df[['FactoryName','FactoryFixedOperatingCost']].assign(**{'FactoryFixedOperatingCost':lambda x: x['FactoryFixedOperatingCost']/12}), on = ['FactoryName'], how = 'outer').fillna(0)\
                    .merge(factories_df[['FactoryName','StorageCapacity','HandlingCapacity']].rename({'HandlingCapacity':'MonthlyHandlingCapacity'},axis=1), on = ['FactoryName'])\
                        .merge(inv_df[['FactoryName','InventoryTurns']].assign(**{'MonthlyInventoryTurns': lambda x: x['InventoryTurns']/12}).drop(['InventoryTurns'],axis=1), on = ['FactoryName'])\
                            .merge(prod_line_policies_df.groupby(['FactoryName']).agg({'Capacity':'sum'}).reset_index().rename({'Capacity':'MonthlyProductionCapacity'},axis=1), on = ['FactoryName'])\
                                .assign(**{'HandlingCapacityUtilization': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])/x['MonthlyHandlingCapacity']})\
                                    .assign(**{'StorageCapacityUtilization': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])/(x['MonthlyInventoryTurns']*x['StorageCapacity'])})\
                                        .assign(**{'ProductionCapacityUtilization': lambda x: x['ProductionQuantity']/x['MonthlyProductionCapacity']})\
                                            .assign(**{'TransportationQuantityOutflow': lambda x: (x['QuantityFactoryToCustomer']+x['QuantityAmongFactoryOutflow'])})\
                                                    .assign(**{'TransportationCost': lambda x: (x['TransportationCostFactoryToCustomer']+x['TransportationCostAmongFactoryInflow'])})
                                                        
factory_cost_monthly_sum_df['EndOfMonthInventory'] = factory_cost_monthly_sum_df.assign(**{'EndOfMonthInventory': lambda x: (x['ProductionQuantity']+x['QuantityAmongFactoryInflow']-x['QuantityAmongFactoryOutflow'])})\
    .sort_values(['FactoryName','Period']).groupby(['FactoryName'])['EndOfMonthInventory'].cumsum()
factory_cost_monthly_sum_df['StartOfMonthInventory'] = factory_cost_monthly_sum_df['EndOfMonthInventory'].shift(1).fillna(0)

In [20]:
### add cost of flow and production
fc_flow_res_df_1 = fc_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination'])\
    .merge(products_df[['ProductName','Volume']], on = ['ProductName'])\
        .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['Price']})


ff_flow_res_df_1 = ff_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination'])\
    .merge(products_df[['ProductName','Volume']], on = ['ProductName'])\
        .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['Price']})

prod_flow_res_df_1 = prod_flow_res_df.merge(factory_var_cost_df, on = ['FactoryName','ProductName'])\
    .assign(**{"FactoryVariableCost": lambda x: x['Quantity']*x['VariableCost']})

### service level info
service_level_df = pd.merge(fc_flow_res_df_1.groupby(['Origin','Period'])\
            .apply(lambda x : pd.Series({'MaxDistanceFactoryToCustomer':x['Distance'].max(),
            'WeightedAverageDistanceFactoryToCustomer':(x['Quantity']*x['Volume']*x['Distance']).sum() / (x['Quantity']*x['Volume']).sum(),
            })).reset_index().rename({'Origin':'FactoryName'}, axis=1),
        ff_flow_res_df_1.groupby(['Destination','Period'])\
            .apply(lambda x : pd.Series({
            'WeightedAverageDistanceAmongFactoryInflow':(x['Quantity']*x['Volume']*x['Distance']).sum() / (x['Quantity']*x['Volume']).sum(),
            })).reset_index().rename({'Destination':'FactoryName'}, axis=1),
        on = ['FactoryName','Period'],
        how = 'outer')

### Save to Excel

In [21]:
### save model output template
writer = pd.ExcelWriter(model_sanitized_path + 'model_output_'+ today +'_'+scenario+'.xlsx', engine='xlsxwriter')

### output data tables
pd.DataFrame({'Scenario': [scenario],
                'TotalCost':[tot_factory_var_cost.getValue()+tot_factory_fixed_cost.getValue()+tot_fc_transp_cost.getValue()+tot_ff_transp_cost.getValue()],
                'FactoryVariableCost':[tot_factory_var_cost.getValue()],
                'FactoryFixedOperatingCost':[tot_factory_fixed_cost.getValue()],
                'TransportationCostFactoryToCustomer':[tot_fc_transp_cost.getValue()],
                'TransportationCostAmongFactoryInflow':[tot_ff_transp_cost.getValue()]})\
        .to_excel(writer, sheet_name='Output_CostSummary', index=False)
factory_cost_sum_df.assign(**{'Scenario': scenario})[['Scenario','FactoryName','FactoryFixedOperatingCost','FactoryVariableCost','TransportationCostFactoryToCustomer','TransportationCostAmongFactoryInflow','ProductionQuantity','AnnualProductionCapacity','ProductionCapacityUtilization','QuantityFactoryToCustomer','QuantityAmongFactoryInflow']]\
        .to_excel(writer, sheet_name='Output_FactoryCostSummary', index=False)
factory_cost_monthly_sum_df.assign(**{'Scenario': scenario})[['Scenario','FactoryName','Period','FactoryFixedOperatingCost','FactoryVariableCost','TransportationCostFactoryToCustomer','TransportationCostAmongFactoryInflow','ProductionQuantity','MonthlyProductionCapacity','ProductionCapacityUtilization','QuantityFactoryToCustomer','QuantityAmongFactoryInflow']]\
        .to_excel(writer, sheet_name='Output_FactoryMonthCostSummary', index=False)
        
fc_flow_res_df_1.assign(**{'Scenario': scenario})[['Scenario','Origin','Destination','ProductName','Period','Quantity','TransportationCost']].to_excel(writer, sheet_name='Output_FactoryToCustomerFlows', index=False)
ff_flow_res_df_1.assign(**{'Scenario': scenario})[['Scenario','Origin','Destination','ProductName','Period','Quantity','TransportationCost']].to_excel(writer, sheet_name='Output_AmongFactoryFlows', index=False)
prod_flow_res_df_1.assign(**{'Scenario': scenario})[['Scenario','FactoryName', 'ProductLine', 'ProductName', 'Period', 'Quantity', 'FactoryVariableCost']].to_excel(writer, sheet_name='Output_FactoryProduction', index=False)
pd.DataFrame({'Scenario': [scenario],
        'TotalDemand':[demand_df['CustomerDemand'].sum()],
        'QuantityFactoryToCustomer':[fc_flow_res_df_1['Quantity'].sum()],
        'DemandSatisfiedPerc': [fc_flow_res_df_1['Quantity'].sum() / demand_df['CustomerDemand'].sum()],
        'WeightedAverageDistanceFactoryToCustomer':[(fc_flow_res_df_1['Quantity']*fc_flow_res_df_1['Volume']*fc_flow_res_df_1['Distance']).sum() / (fc_flow_res_df_1['Quantity']*fc_flow_res_df_1['Volume']).sum()],
        'QuantityAmongFactoryInflow':[ff_flow_res_df_1['Quantity'].sum()],
        'WeightedAverageDistanceAmongFactoryInflow':[(ff_flow_res_df_1['Quantity']*ff_flow_res_df_1['Volume']*ff_flow_res_df_1['Distance']).sum() / (ff_flow_res_df_1['Quantity']*ff_flow_res_df_1['Volume']).sum()],
        'MaxDistanceFactoryToCustomer':[fc_flow_res_df_1['Distance'].max()]})\
                .to_excel(writer, sheet_name='Output_ServiceLevel', index=False)

inv_level_res_df.assign(**{'Scenario': scenario})[['Scenario','FactoryName','ProductName','Period','Quantity']].to_excel(writer, sheet_name='Output_InventoryLevel', index=False)

### need further cleaning format before send to client
### input data tables
products_df.to_excel(writer, sheet_name='Input_ProductList', index=False)
customers_df.to_excel(writer, sheet_name='Input_CustomerList', index=False)
factories_df.to_excel(writer, sheet_name='Input_FactoryList', index=False)
factory_var_cost_df.to_excel(writer, sheet_name='Input_FactoryVariableCost', index=False)
factory_fixed_cost_df.to_excel(writer, sheet_name='Input_FactoryFixedCost', index=False)
trans_policies_df.to_excel(writer, sheet_name='Input_TransportationPolicy', index=False)
prod_policies_df.to_excel(writer, sheet_name='Input_CapacityByLineByProduct', index=False)
prod_line_policies_df.to_excel(writer, sheet_name='Input_CapacityByLine', index=False)
inv_df.to_excel(writer, sheet_name='Input_InventoryPolicy', index=False)
demand_df.to_excel(writer, sheet_name='Input_CustomerDemand', index=False)
hist_outbound_trans_df.to_excel(writer, sheet_name='Input_HistShipmentFactoryToCust', index=False)
hist_inbound_trans_df.to_excel(writer, sheet_name='Input_HistShipmentAmongFactory', index=False)
hist_prod_df.to_excel(writer, sheet_name='Input_HistProduction', index=False)
hist_inv_df.to_excel(writer, sheet_name='Input_HistInventoryLevel', index=False)

writer.save()

In [22]:
time.time()

1655295201.3283813